<a href="https://colab.research.google.com/github/adithyamauryakr/pytorchtutorials/blob/main/optuna-hpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Grid Search CV is not scalable. Random Search is simple but can overlook better solutions.
Optuna solves these problem > Bayesian search.
* find a mathematical relation between tunable parameters and accuracy to be estimated. Hence through this we can the best possible parameters.

## Key Terms of optuna

1. Study: optimization session encompassing multiple trials.
2. Trial: different combinations used to train a model
3. Trial parameters: Hyperparameter values of a trial
4. Objective funciton: relationship between parameters and accuracy.
5. Sampler: suggests what HP to tryout next. Optuna uses Tree-structured Parzen Estimator (TPE)

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.8 MB/s eta 0:00:00


In [2]:
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
import numpy as np
# replace zero with NaN in columns where zero is not a valid value
cols_with_missing_vals=['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals]=df[cols_with_missing_vals].replace(0, np.nan)

# impute missing vals with the mean of the respective column
df.fillna(df.mean(), inplace=True)

print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [5]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f'Training set shap: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shap: (614, 8)
Test set shape: (154, 8)


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define objective function
def objective(trail):
  # Suggest vals of hyperparameters:
  n_estimators = trail.suggest_int('n_estimators', 50, 200)
  max_depth = trail.suggest_int('max_depth', 3, 20)

  # create RandomForestClassifier with suggest HP
  model = RandomForestClassifier(
      n_estimators=n_estimators,
      max_depth=max_depth,
      random_state=42
  )
  score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
  return score # return the accuracy score for Optuna to maximize

In [9]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=50)

[I 2025-03-20 14:26:00,307] A new study created in memory with name: no-name-d8be51d3-53ce-4f37-9777-21a12eba1eff
[I 2025-03-20 14:26:01,520] Trial 0 finished with value: 0.7752112227004623 and parameters: {'n_estimators': 146, 'max_depth': 10}. Best is trial 0 with value: 0.7752112227004623.
[I 2025-03-20 14:26:02,390] Trial 1 finished with value: 0.771967160848079 and parameters: {'n_estimators': 133, 'max_depth': 6}. Best is trial 0 with value: 0.7752112227004623.
[I 2025-03-20 14:26:03,136] Trial 2 finished with value: 0.7703331739199745 and parameters: {'n_estimators': 153, 'max_depth': 17}. Best is trial 0 with value: 0.7752112227004623.
[I 2025-03-20 14:26:03,427] Trial 3 finished with value: 0.7768691216323927 and parameters: {'n_estimators': 61, 'max_depth': 15}. Best is trial 3 with value: 0.7768691216323927.
[I 2025-03-20 14:26:04,318] Trial 4 finished with value: 0.7736011477761836 and parameters: {'n_estimators': 184, 'max_depth': 18}. Best is trial 3 with value: 0.7768691

In [10]:
print(f'Best trial acc: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial acc: 0.7850071736011478
Best hyperparameters: {'n_estimators': 143, 'max_depth': 13}


In [11]:
from sklearn.metrics import accuracy_score

best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)

print(f'Test accuracy with best hyperparameters: {test_accuracy:.2f}')

Test accuracy with best hyperparameters: 0.75


## Sampler in optuna

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define objective function
def objective(trail):
  # Suggest vals of hyperparameters:
  n_estimators = trail.suggest_int('n_estimators', 50, 200)
  max_depth = trail.suggest_int('max_depth', 3, 20)

  # create RandomForestClassifier with suggest HP
  model = RandomForestClassifier(
      n_estimators=n_estimators,
      max_depth=max_depth,
      random_state=42
  )
  score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
  return score # return the accuracy score for Optuna to maximize

In [13]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())
study.optimize(objective, n_trials=50)

[I 2025-03-20 14:34:59,090] A new study created in memory with name: no-name-68a43549-bca8-4ff8-945d-611a1a923f63
[I 2025-03-20 14:35:00,514] Trial 0 finished with value: 0.7882751474573569 and parameters: {'n_estimators': 122, 'max_depth': 13}. Best is trial 0 with value: 0.7882751474573569.
[I 2025-03-20 14:35:01,212] Trial 1 finished with value: 0.771975131516021 and parameters: {'n_estimators': 80, 'max_depth': 20}. Best is trial 0 with value: 0.7882751474573569.
[I 2025-03-20 14:35:01,819] Trial 2 finished with value: 0.7752351347042882 and parameters: {'n_estimators': 73, 'max_depth': 17}. Best is trial 0 with value: 0.7882751474573569.
[I 2025-03-20 14:35:03,247] Trial 3 finished with value: 0.7670731707317073 and parameters: {'n_estimators': 193, 'max_depth': 6}. Best is trial 0 with value: 0.7882751474573569.
[I 2025-03-20 14:35:04,601] Trial 4 finished with value: 0.7801052128168341 and parameters: {'n_estimators': 180, 'max_depth': 14}. Best is trial 0 with value: 0.78827514

In [14]:
print(f'Best trial acc: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial acc: 0.7882751474573569
Best hyperparameters: {'n_estimators': 122, 'max_depth': 13}


In [15]:
from sklearn.metrics import accuracy_score

best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)

print(f'Test accuracy with best hyperparameters: {test_accuracy:.2f}')

Test accuracy with best hyperparameters: 0.76


In [16]:
# Grid search in optuna
screen_space =  {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [19]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(screen_space))
study.optimize(objective, n_trials=50)

[I 2025-03-20 14:38:29,242] A new study created in memory with name: no-name-8092bc3b-db92-4dc2-bc95-d7ca8e634550
[I 2025-03-20 14:38:30,027] Trial 0 finished with value: 0.7654391838036028 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7654391838036028.
[I 2025-03-20 14:38:31,138] Trial 1 finished with value: 0.7735772357723577 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 1 with value: 0.7735772357723577.
[I 2025-03-20 14:38:31,667] Trial 2 finished with value: 0.7687151283277539 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 1 with value: 0.7735772357723577.
[I 2025-03-20 14:38:32,507] Trial 3 finished with value: 0.7752351347042882 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 3 with value: 0.7752351347042882.
[I 2025-03-20 14:38:33,345] Trial 4 finished with value: 0.7703491152558585 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 3 with value: 0.775235